In [13]:
import onnxruntime as ort
import numpy as np
import scipy.special
from PIL import Image


# 预处理图像
def preprocess_image(image, resize_size=256, crop_size=224, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    image = image.resize((resize_size, resize_size), Image.BILINEAR)
    w, h = image.size
    left = (w - crop_size) / 2
    top = (h - crop_size) / 2
    image = image.crop((left, top, left + crop_size, top + crop_size))
    image = np.array(image).astype(np.float32)
    image = image / 255.0
    image = (image - mean) / std
    image = np.transpose(image, (2, 0, 1))
    image = image.reshape((1,) + image.shape)
    return image


# 加载模型  2分
session = ort.InferenceSession('flower-detection.onnx')


# 加载类别标签 2分
with open('labels.txt') as f:
    labels = [line.strip() for line in f.readlines()]


# 获取模型输入和输出的名称
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name


# 加载图片  2分
image = Image.open('flower_test.png').convert('RGB')


# 预处理图片  2分
processed_image = preprocess_image(image)


# 确保输入数据是 float32 类型
processed_image = processed_image.astype(np.float32)

# 进行图片识别  2分
output = session.run([output_name], {input_name: processed_image})[0]


# 应用 softmax 函数获取识别分类后的准确率  2分
accuracy = scipy.special.softmax(output, axis=-1)
print(accuracy)

# 获取预测的类别索引
predicted_idx =  np.argmax(output[0])

# 获取预测的准确值（转换为百分比）
prob_percentage = accuracy[0, predicted_idx]*100

print(accuracy[0, predicted_idx])

# 获取预测的类别标签
predicted_label = labels[predicted_idx]


# 输出预测结果，包含百分比形式的概率
print(f"Predicted class: {predicted_label}, Accuracy: {prob_percentage:.2f}%")


[[2.39586143e-06 2.93439753e-05 8.90261305e-08 2.50166057e-08
  2.15281915e-07 7.72698684e-07 5.03510691e-08 4.33365130e-06
  3.11128633e-06 2.87396034e-07 2.06182212e-05 4.62673343e-05
  7.48302318e-06 1.99655688e-06 1.14799666e-04 7.97593748e-06
  2.71746612e-05 5.49619790e-06 5.08169751e-06 7.89828300e-06
  8.94737514e-06 1.51143422e-06 6.13680129e-07 1.63025732e-06
  2.02332126e-06 7.95758115e-06 2.86248578e-05 1.29586506e-05
  2.31118702e-05 5.93644518e-06 5.85330599e-06 3.12638549e-05
  1.10922638e-05 1.14902456e-07 6.18383581e-07 1.51623021e-06
  1.20154509e-05 2.36067262e-06 1.56659837e-06 1.53665235e-06
  1.54121299e-05 3.22094161e-06 3.11924236e-06 1.86572800e-06
  3.47825016e-06 7.89827141e-07 1.05795380e-05 6.15834415e-06
  1.29053106e-07 1.69857898e-07 3.51939320e-07 2.16287191e-07
  1.41030041e-05 2.08493093e-05 1.76256026e-06 4.61562513e-06
  3.04144010e-06 3.70714656e-06 4.65101448e-06 3.42444109e-05
  4.25206872e-06 5.64949744e-07 1.45854904e-06 1.19760227e-06
  2.3803